In [1]:
# Assistant API(OpenAI)
# Agent와 비슷한 기능 -> Langchain 없이 OpenAI Python 패키지로 가능
"""
1. Assistant
 : Tool(Instructino) 과 OpenAI Model 로 조합된 AI Model
2. Thread
 : User와 Assistant가 주고 받는 메시지의 모음
 : User가 Message를 보내면 Thread에 전달되야 함.
 : 별도 메모리 관리를 할 것없이 OpenAI가 이를 관리하여 Message들을 Thread에 관리됨(Model의 context memory에 맞도록 history 관리)
3. Messages
 : User Message
4. Run 
 : Thread의 내용을 Model을 통해서 실행
"""
from langchain.tools import DuckDuckGoSearchResults
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchResults()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


In [2]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview", # https://platform.openai.com/playground?mode=assistant : Model List
#     tools=functions,
# )

assistant_id = "asst_HUHCXk0HEeHfarAzTKrt5f50"

In [3]:
# 1. Thread 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_9KczPqLkubVd9QifPKIBVusi', created_at=1712846058, metadata={}, object='thread')

In [21]:
# 2. Run 생성
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_beFmhw6BsJirNXKhzf49OBPd', assistant_id='asst_HUHCXk0HEeHfarAzTKrt5f50', cancelled_at=None, completed_at=None, created_at=1712846719, expires_at=1712847319, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_9KczPqLkubVd9QifPKIBVusi', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type':

In [6]:
# Run 실행
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

# Message를 입력된 순서대로 보여주게 Reverse
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        # print(message)
        print(f"{message.role}: {message.content[0].text.value}")

# run에 사용할 tool를 찾고 실행
def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs

def submit_tool_outputs(run_id, thread_id):
    # Output을 Submit
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outputs
    )

In [10]:
# 3. run 실행 -> next action
"""
requires_action - assistant 가 호출하길 바라는 function을 실행시켜 줘야 함.
"""
get_run(run.id, thread.id).status

'requires_action'

In [11]:
# 3-1. message 확인
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy


In [12]:
# 4. Next Action
submit_tool_outputs(run.id, thread.id)

Calling function: get_ticker with arg {"company_name":"Salesforce"}


Run(id='run_Ie3LbvuOlnSyzdcwSLtsh39K', assistant_id='asst_HUHCXk0HEeHfarAzTKrt5f50', cancelled_at=None, completed_at=None, created_at=1712846072, expires_at=1712846672, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1712846072, status='queued', thread_id='thread_9KczPqLkubVd9QifPKIBVusi', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'

In [13]:
get_run(run.id, thread.id).status

'requires_action'

In [14]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy


In [ ]:
# 24.04.11 버전에서는 Playground에서 첫 Thread를 줬을 때 부터 Requires Action 상태가 됨.(get_ticker)
# send_message(thread.id, "Please go ahead!")

In [15]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_Ie3LbvuOlnSyzdcwSLtsh39K', assistant_id='asst_HUHCXk0HEeHfarAzTKrt5f50', cancelled_at=None, completed_at=None, created_at=1712846072, expires_at=1712846672, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1712846089, status='queued', thread_id='thread_9KczPqLkubVd9QifPKIBVusi', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'

In [17]:
get_run(run.id, thread.id).status

'completed'

In [18]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: I have gathered some fundamental and recent stock performance data on Salesforce (CRM) to help assess whether it may be a good buy. Please remember that investment decisions should take into account a variety of factors, including your individual circumstances, investment goals, and risk tolerance. Here is the data:

Income Statement Highlights (most recent data):
- Total Revenue: \$34.857 billion
- Net Income: \$4.136 billion
- Gross Profit: \$26.316 billion
- Research & Development: \$4.906 billion
- Selling, General & Administrative: \$14.52 billion

Balance Sheet Highlights (most recent data):
- Total Assets: \$99.823 billion
- Total Liabilities: \$40.177 billion
- Total Equity: \$59.646 billion
- Total Debt: \$12.588 billion
- Working Capital: \$2.443 billion

Stock Performance Over the Last 100 Days:
- The stock has seen fluctuations with highs and lows, like most stocks in the market. Overall, without exact pe

In [19]:
send_message(thread.id, "Now I want to know if the Cloudflare is a good buy.")

Message(id='msg_XscOFYOS1Ijq5MrOyDiISLvU', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Now I want to know if the Cloudflare is a good buy.'), type='text')], created_at=1712846645, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_9KczPqLkubVd9QifPKIBVusi')

In [20]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: I have gathered some fundamental and recent stock performance data on Salesforce (CRM) to help assess whether it may be a good buy. Please remember that investment decisions should take into account a variety of factors, including your individual circumstances, investment goals, and risk tolerance. Here is the data:

Income Statement Highlights (most recent data):
- Total Revenue: \$34.857 billion
- Net Income: \$4.136 billion
- Gross Profit: \$26.316 billion
- Research & Development: \$4.906 billion
- Selling, General & Administrative: \$14.52 billion

Balance Sheet Highlights (most recent data):
- Total Assets: \$99.823 billion
- Total Liabilities: \$40.177 billion
- Total Equity: \$59.646 billion
- Total Debt: \$12.588 billion
- Working Capital: \$2.443 billion

Stock Performance Over the Last 100 Days:
- The stock has seen fluctuations with highs and lows, like most stocks in the market. Overall, without exact pe

In [22]:
# run을 다시 생성 하고, 
get_run(run.id, thread.id).status

'requires_action'

In [23]:
# requires_action에서 필요한 function(tool) 확인
get_tool_outputs(run.id, thread.id)

Calling function: get_ticker with arg {"company_name":"Cloudflare"}


[{'output': "[snippet: Real-time Cloudflare stock price (NYSE: NET) plus other valuable data points like day range, year, stock analyst insights, related news and more., title: NET Stock - Cloudflare Stock Price - NYSE: NET | Morningstar, link: https://www.morningstar.com/stocks/xnys/net/quote], [snippet: At Cloudflare, Inc., we promise to treat your data with respect and will not share your information with any third party. You can unsubscribe to any of the investor alerts you are subscribed to by visiting the 'unsubscribe' section below. If you experience any issues with this process, please contact us for further assistance., title: Cloudflare, Inc. - Stock Quote & Chart, link: https://cloudflare.net/stock-information/Stock-Quote--Chart/], [snippet: Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile., title: NET: Cloudflare Inc Stock Price Quote - New York - Bloomberg, link: https://www.

In [24]:
# Tool 실행
submit_tool_outputs(run.id, thread.id)

Calling function: get_ticker with arg {"company_name":"Cloudflare"}


Run(id='run_beFmhw6BsJirNXKhzf49OBPd', assistant_id='asst_HUHCXk0HEeHfarAzTKrt5f50', cancelled_at=None, completed_at=None, created_at=1712846719, expires_at=1712847319, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1712846720, status='queued', thread_id='thread_9KczPqLkubVd9QifPKIBVusi', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'

In [25]:
get_run(run.id, thread.id).status

'requires_action'

In [26]:
get_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "NET"}
Calling function: get_balance_sheet with arg {"ticker": "NET"}
Calling function: get_daily_stock_performance with arg {"ticker": "NET"}


[{'output': '"{\\"1703980800000\\":{\\"Tax Effect Of Unusual Items\\":-10563000.0,\\"Tax Rate For Calcs\\":0.21,\\"Normalized EBITDA\\":14130000.0,\\"Total Unusual Items\\":-50300000.0,\\"Total Unusual Items Excluding Goodwill\\":-50300000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":-183949000.0,\\"Reconciled Depreciation\\":135820000.0,\\"Reconciled Cost Of Revenue\\":307005000.0,\\"EBITDA\\":-36170000.0,\\"EBIT\\":-171990000.0,\\"Net Interest Income\\":62295000.0,\\"Interest Expense\\":5872000.0,\\"Interest Income\\":68167000.0,\\"Normalized Income\\":-144212000.0,\\"Net Income From Continuing And Discontinued Operation\\":-183949000.0,\\"Total Expenses\\":1482230000.0,\\"Total Operating Income As Reported\\":-185485000.0,\\"Diluted Average Shares\\":333656000.0,\\"Basic Average Shares\\":333656000.0,\\"Diluted EPS\\":-0.55,\\"Basic EPS\\":-0.55,\\"Diluted NI Availto Com Stockholders\\":-183949000.0,\\"Net Income Common Stockholders\\":-183949000.0,\\"Net Incom

In [27]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "NET"}
Calling function: get_balance_sheet with arg {"ticker": "NET"}
Calling function: get_daily_stock_performance with arg {"ticker": "NET"}


Run(id='run_beFmhw6BsJirNXKhzf49OBPd', assistant_id='asst_HUHCXk0HEeHfarAzTKrt5f50', cancelled_at=None, completed_at=None, created_at=1712846719, expires_at=1712847319, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1712846855, status='queued', thread_id='thread_9KczPqLkubVd9QifPKIBVusi', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'

In [29]:
get_run(run.id, thread.id).status

'completed'

In [30]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: I have gathered some fundamental and recent stock performance data on Salesforce (CRM) to help assess whether it may be a good buy. Please remember that investment decisions should take into account a variety of factors, including your individual circumstances, investment goals, and risk tolerance. Here is the data:

Income Statement Highlights (most recent data):
- Total Revenue: \$34.857 billion
- Net Income: \$4.136 billion
- Gross Profit: \$26.316 billion
- Research & Development: \$4.906 billion
- Selling, General & Administrative: \$14.52 billion

Balance Sheet Highlights (most recent data):
- Total Assets: \$99.823 billion
- Total Liabilities: \$40.177 billion
- Total Equity: \$59.646 billion
- Total Debt: \$12.588 billion
- Working Capital: \$2.443 billion

Stock Performance Over the Last 100 Days:
- The stock has seen fluctuations with highs and lows, like most stocks in the market. Overall, without exact pe